In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://github.com/PacktWorkshops/The-Deep-Learning-Workshop/raw/master/Chapter03/Datasets/Activity3.02/fruits360.zip'

In [3]:
zip_dir = tf.keras.utils.get_file('fruits360.zip', origin=file_url, extract=True)

82231296/82220233 [==============================] - 6s 0us/step


In [4]:
import pathlib

In [5]:
path = pathlib.Path(zip_dir).parent / 'fruits360_filtered'

In [6]:
train_dir = path / 'Training'
validation_dir = path / 'Test'

In [7]:
total_train = 11398
total_val = 4752

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
train_image_generator = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

In [10]:
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [11]:
batch_size=16
img_height = 100
img_width = 100
channel = 3

In [12]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           target_size=(img_height, img_width))

Found 11398 images belonging to 120 classes.


In [13]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(img_height, img_width))

Found 4752 images belonging to 120 classes.


In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [15]:
np.random.seed(8)
tf.random.set_seed(8)

In [16]:
from tensorflow.keras.applications import VGG16

In [17]:
base_model = VGG16(input_shape=(img_height, img_width, channel), weights='imagenet', include_top=False)

2022-05-28 08:55:11.467146: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
base_model.trainable =  False

In [19]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [20]:
model = tf.keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dense(120, activation='softmax')
])

In [21]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [22]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 3, 3, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 1000)              4609000   
                                                                 
 dense_1 (Dense)             (None, 120)               120120    
                                                                 
Total params: 19,443,808
Trainable params: 4,729,120
Non-trainable params: 14,714,688
_________________________________________________________________


In [24]:
model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5


/var/folders/fm/b49y7nm968107m07lbt3cqsc0000gp/T/ipykernel_31259/793859307.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


712/712 [==============================] - 328s 460ms/step - loss: 2.0414 - accuracy: 0.5062 - val_loss: 1.0203 - val_accuracy: 0.7022
Epoch 2/5
712/712 [==============================] - 375s 527ms/step - loss: 0.6772 - accuracy: 0.8054 - val_loss: 0.6821 - val_accuracy: 0.7891
Epoch 3/5
712/712 [==============================] - 385s 540ms/step - loss: 0.4424 - accuracy: 0.8610 - val_loss: 0.5205 - val_accuracy: 0.8493
Epoch 4/5
712/712 [==============================] - 381s 535ms/step - loss: 0.3340 - accuracy: 0.8965 - val_loss: 0.4549 - val_accuracy: 0.8638
Epoch 5/5
712/712 [==============================] - 379s 533ms/step - loss: 0.3079 - accuracy: 0.9027 - val_loss: 0.3962 - val_accuracy: 0.8883
